In [384]:
import pyomo.environ as pyo
import numpy as np

In [385]:
model = pyo.ConcreteModel()


# a = np.array([[1,0,1,1,0,0,0,1,0,1,0,1],
#      [1,1,0,0,0,0,0,0,0,1,0,0],
#      [1,0,1,0,1,0,1,0,1,0,1,0],
#      [1,1,0,0,1,0,1,0,0,1,1,0],
#      [1,0,1,1,0,0,1,0,0,0,0,1],
#      [1,1,0,0,0,1,1,0,0,0,1,0],
#      [1,0,1,0,0,1,1,0,0,1,1,0],
#      [0,0,0,1,0,0,0,0,0,0,1,0]])

a = np.array(
    [[1,0,1,0,0,0,0],
     [0,1,1,0,1,0,1],
     [0,1,1,0,0,1,1],
     [0,0,0,1,0,0,1]])

# 12 courses
C = [i for i in range(a.shape[1])]
model.add_component('C', pyo.Set(initialize=C))
# Days: Saturday:0 Sunday:1 Monday:2
D = [0,1]
model.add_component('D', pyo.Set(initialize=D))
# Time slots: (8_10):0  (10_12):1
H=[0,1]
model.add_component('H', pyo.Set(initialize=H))
# Rooms: A:0  B:1
R=[0,1]
model.add_component('R', pyo.Set(initialize=R))
# 8 students
S = [i for i in range(a.shape[0])]
model.add_component('S', pyo.Set(initialize=S))


model.add_component('a', pyo.Param(model.S,model.C, rule=lambda model,s,c: a[s][c]))
# capacity: A:3 B:4
cap=[1,2]
model.add_component('cap', pyo.Param(model.R, initialize=cap))
# last day
model.add_component('lastDay', pyo.Param(initialize=D.pop()))


model.add_component('delta', pyo.Var(model.C,model.D,model.H,model.R, domain=pyo.Binary))
model.add_component('x', pyo.Var(model.C,model.D,model.H,model.R, domain=pyo.NonNegativeIntegers))
model.add_component('psi', pyo.Var(model.S,model.C,model.D,model.H, domain=pyo.Binary))
model.add_component('gamma', pyo.Var(model.S,model.D,model.H, domain=pyo.Binary))
model.add_component('sigma', pyo.Var(model.S,model.D,model.D, domain=pyo.Binary))
model.add_component('landa', pyo.Var(model.S,model.D,model.H, domain=pyo.NonNegativeReals))
model.add_component('landa_prim', pyo.Var(model.S,model.D,model.H, domain=pyo.NonNegativeReals))
model.add_component('beta', pyo.Var(model.S,model.D,model.D, domain=pyo.NonNegativeReals))
model.add_component('beta_prim', pyo.Var(model.S,model.D,model.D, domain=pyo.NonNegativeReals))
model.add_component('zeta', pyo.Var(model.D,model.H,model.R, domain=pyo.NonNegativeReals))

---
### Objective Function

$$\begin{equation}\begin{aligned} min \;\; z & = \sum_{s,d,h}U_{s,d,h}\cdot\lambda_{s,d,h}
 +\sum_{s,d,h}U_{s,d,h}^\prime\cdot\lambda_{s,d,h}^\prime \\\\ &
 +\sum_{s,d,d^\prime}V_{s,d,d^\prime}\cdot\beta_{s,d,d^\prime} +\sum_{s,d,d^\prime}V_{s,d,d^\prime}^\prime\cdot\beta_{s,d,d^\prime}^\prime \\\\ &
 +\sum_{d,h,r}W_{d,h,r}\cdot\zeta_{d,h,r}
 \end{aligned}\end{equation}$$

In [386]:
def objective_rule(model):
    res  = 0.8*sum(model.landa[s,d,h] for s in model.S for d in model.D for h in model.H)
    res += 0.8*sum(model.landa_prim[s,d,h] for s in model.S for d in model.D for h in model.H)
    res += 0.5*sum(model.beta[s,d,d] for s in model.S for d in model.D for d in model.D)
    res += 0.5*sum(model.beta_prim[s,d,d] for s in model.S for d in model.D for d in model.D)
    res += 0.2*sum(model.zeta[d,h,r] for d in model.D for h in model.H for r in model.R)
    return res
model.add_component('objective_function', pyo.Objective(rule=objective_rule, sense=pyo.minimize))

---
### Hard Constraint 1

$$\sum_{d,h,r} \delta_{c,d,h,r} \geq 1 \hspace{1cm} \forall c$$
$$\delta_{c,d,h,r} = 1\implies\sum_{d^\prime, h^\prime, r^\prime : (d,h)\neq(d^\prime, h^\prime)} \delta_{c,d^\prime,h^\prime, r^\prime} \leq 0 \hspace{1cm} \forall s,c,d,h,r$$

##### Linearized:
Part 1:
$$\sum_{d,h,r} \delta_{c,d,h,r} \geq 1 \hspace{1cm} \forall c$$
Part 2:
$$\sum_{d^\prime, h^\prime, r^\prime : (d,h)\neq(d^\prime, h^\prime)} \delta_{c,d^\prime,h^\prime, r^\prime} \leq |\mathbb{D \times H \times R}|\cdot(1-\delta_{c,d,h,r})\hspace{1cm} \forall s,c,d,h,r$$

In [387]:
def hard1_part1_rule(model,c):
   return sum(model.delta[c,d,h,r] for d in model.D for h in model.H for r in model.R) >= 1
model.add_component('hard1_part1', pyo.Constraint(model.C, rule=hard1_part1_rule))

def hard1_part2_rule(model,c,d,h,r):
  M = len(model.D)*len(model.H)*len(model.R)
  return sum(model.delta[c,d_prim,h_prim,r_prim] for d_prim in model.D for h_prim in model.H \
             for r_prim in model.R if d_prim!=d or h_prim!=h) \
             <= M * (1 - model.delta[c,d,h,r])
model.add_component('hard1_part2', pyo.Constraint(model.C,model.D,model.H,model.R, rule=hard1_part2_rule))

---
### Hard Constraint 2

$$\sum_{r}a_{s,c}\cdot\delta_{c,d,h,r} \geq 1 \implies \sum_{c^\prime,r \; \colon \; c \neq c^\prime} a_{s,c^\prime}\cdot\delta_{c^\prime,d,h,r} \leq 0 \hspace{1cm} \forall s,c,d,h$$

##### Linearized:
Part 1:
$$\sum_{r}a_{s,c}\cdot\delta_{c,d,h,r} \leq \|\mathbb{R}\|\cdot \psi_{s,c,d,h} \hspace{1cm} \forall s,c,d,h$$

Part 2:
$$\sum_{c^\prime,r \; \colon \; c \neq c^\prime} a_{s,c^\prime}\cdot\delta_{c^\prime,d,h,r} \leq\|\mathbb{C\times R}\|\cdot (1-\psi_{s,c,d,h}) \hspace{1cm} \forall s,c,d,h$$

In [388]:
def hard2_part1_rule(model,s,c,d,h):
  M = len(model.R)
  return sum(model.a[s,c]*model.delta[c,d,h,r] for r in model.R) <= M*model.psi[s,c,d,h]
model.add_component('hard2_part1', pyo.Constraint(model.S,model.C,model.D,model.H, rule=hard2_part1_rule))

def hard2_part2_rule(model,s,c,d,h):
  M = len(model.C)*len(model.R)
  return sum(model.a[s,c_prim]*model.delta[c_prim,d,h,r] for r in model.R for c_prim in model.C if c_prim!=c) \
        <= M*(1-model.psi[s,c,d,h])
model.add_component('hard2_part2', pyo.Constraint(model.S,model.C,model.D,model.H, rule=hard2_part2_rule))

---
### Hard Constraint 4:

$$\sum_{c}x_{c,d,h,r} \leq Cap_r \hspace{1cm} \forall d,h,r$$

##### Variable Relations:
$$\delta_{c,d,h,r} \geq 1 \implies x_{c,d,h,r} \geq 1 \hspace{1cm} \forall c,d,h,r$$
$$\delta_{c,d,h,r} \leq 0 \implies x_{c,d,h,r} \leq 0 \hspace{1cm} \forall c,d,h,r$$
$$\sum_{d,h,r}x_{c,d,h,r} = \sum_{s} a_{s,c} \hspace{1cm} \forall c \hspace{0.3cm}$$

In [389]:
def hard4_part1_rule(model,d,h,r):
  return sum(model.x[c,d,h,r] for c in model.C) <= model.cap[r]
model.add_component('hard4_part1', pyo.Constraint(model.D,model.H,model.R, rule=hard4_part1_rule))

#variable relations
def hard4_part2_rule(model,c,d,h,r):
  M = 1
  return model.x[c,d,h,r] >= 1 - M * (1- model.delta[c,d,h,r])
model.add_component('hard4_part2', pyo.Constraint(model.C,model.D,model.H,model.R, rule=hard4_part2_rule))

def hard4_part3_rule(model,c,d,h,r):
  M =  model.cap[r]
  return model.x[c,d,h,r] <= M * model.delta[c,d,h,r]
model.add_component('hard4_part3', pyo.Constraint(model.C,model.D,model.H,model.R, rule=hard4_part3_rule))

def hard4_part4_rule(model,c):
  return sum(model.x[c,d,h,r] for d in model.D for h in model.H for r in model.R) \
         == sum(model.a[s,c] for s in model.S)
model.add_component('hard4_part4', pyo.Constraint(model.C, rule=hard4_part4_rule))

---
### Soft Constraint 1:

$$\sum_{r, c} a_{s,c}\cdot \delta_{c,d,h,r} \geq 1 \implies \sum_{r, c, h^\prime \neq h} a_{s,c}\cdot \delta_{c,d,h^\prime,r} \leq 0 \hspace{1cm} \forall s,d,h$$

##### Linearized:

$$\sum_{r, c} a_{s,c}\cdot \delta_{c,d,h,r} - \lambda_{s,d,h} \leq  \|\mathbb{R\times C}\|\cdot \gamma_{s,d,h} \hspace{1cm} \forall s,d,h$$
$$\sum_{r, c, h^\prime \neq h} a_{s,c}\cdot \delta_{c,d,h^\prime,r} - \lambda_{s,d,h}^\prime\leq \|\mathbb{C\times R \times H\|}\cdot (1-\gamma_{s,d,h}) \hspace{1cm} \forall s,d,h$$

In [390]:
def soft1_part1_rule(model,s,d,h):
  M = len(model.C)*len(model.R)
  return sum(model.a[s,c]*model.delta[c,d,h,r] for r in model.R for c in model.C) \
        - model.landa[s,d,h] <= M * model.gamma[s,d,h]
model.add_component('soft1_part1', pyo.Constraint(model.S,model.D,model.H, rule=soft1_part1_rule))

def soft1_part2_rule(model,s,d,h):
  M = len(model.C)*len(model.R)*len(model.H)
  return sum(model.a[s,c]*model.delta[c,d,h_prim,r] for r in model.R for c in model.C for h_prim in model.H if h_prim!=h) \
        - model.landa_prim[s,d,h] <= M * (1 - model.gamma[s,d,h])
model.add_component('soft1_part2', pyo.Constraint(model.S,model.D,model.H, rule=soft1_part2_rule))

---
### [OPTIONAL] Soft Constraint 1 (EXTRAS):
$$\lambda_{s,d,h}>0 \implies \sum_{h^\prime \neq h}\lambda_{s,d,h^\prime} \leq 0 \hspace{1cm}\forall s,d,h$$
$$\lambda_{s,d,h}^\prime>0 \implies \sum_{h^\prime \neq h}\lambda_{s,d,h^\prime}^\prime \leq 0 \hspace{1cm}\forall s,d,h$$

##### Linearized:

$$\lambda_{s,d,h}\leq\|\mathbb{R\times C}\|\cdot\omega_{s,d,h}\hspace{1cm}\forall s,d,h$$
$$\sum_{h^\prime \neq h}\lambda_{s,d,h^\prime} \leq \|\mathbb{H}\|\cdot\|\mathbb{R\times C}\|\cdot(1-\omega_{s,d,h}) \hspace{1cm}\forall s,d,h$$
$$\lambda_{s,d,h}^\prime\leq\|\mathbb{R\times C}\|\cdot\omega_{s,d,h}^\prime \hspace{1cm}\forall s,d,h$$
$$\sum_{h^\prime \neq h}\lambda_{s,d,h^\prime}^\prime \leq \|\mathbb{H}\|\cdot\|\mathbb{R\times C}\|\cdot(1-\omega_{s,d,h}^\prime) \hspace{1cm}\forall s,d,h$$

> Running the following block is optional. It will reduce the optimality of the model but will increase the correlation to the original problem.

In [391]:
model.add_component('omega', pyo.Var(model.S,model.D,model.H, domain=pyo.Binary))
model.add_component('omega_prim', pyo.Var(model.S,model.D,model.H, domain=pyo.Binary))


def soft1_extra_part1_rule(model,s,d,h):
  M = len(model.R)*len(model.C)
  return model.landa[s,d,h] <= M*model.omega[s,d,h]
model.add_component('soft1_extra_part1', pyo.Constraint(model.S,model.D,model.H, rule=soft1_extra_part1_rule))

def soft1_extra_part2_rule(model,s,d,h):
  M = len(model.H)*len(model.R)*len(model.C)
  return sum(model.landa[s,d,h_prim] for h_prim in model.H if h_prim!=h) <= M * (1-model.omega[s,d,h])
model.add_component('soft1_extra_part2', pyo.Constraint(model.S,model.D,model.H, rule=soft1_extra_part2_rule))

def soft1_extra_part3_rule(model,s,d,h):
  M = len(model.R)*len(model.C)
  return model.landa_prim[s,d,h] <= M*model.omega_prim[s,d,h]
model.add_component('soft1_extra_part3', pyo.Constraint(model.S,model.D,model.H, rule=soft1_extra_part3_rule))

def soft1_extra_part4_rule(model,s,d,h):
  M = len(model.H)*len(model.R)*len(model.C)
  return sum(model.landa_prim[s,d,h_prim] for h_prim in model.H if h_prim!=h) <= M * (1-model.omega_prim[s,d,h])
model.add_component('soft1_extra_part4', pyo.Constraint(model.S,model.D,model.H, rule=soft1_extra_part4_rule))




---
### Soft Constraint 2

$$\sum_{c,h,r} a_{s,c}\cdot \delta_{c,d,h,r} \geq 1 \implies \sum_{c, h,r} a_{s,c}\cdot \delta_{c,d^\prime,h,r} \leq 0 \hspace{1cm} \forall s,d,d^\prime$$

##### Linearized:

$$\sum_{c,h,r} a_{s,c}\cdot \delta_{c,d,h,r} - \beta_{s,d,d^\prime} \leq 0 + \|\mathbb{C\times R \times H\|}\cdot \sigma_{s,d,d^\prime}\hspace{1cm} \forall s,d,d^\prime$$
$$\sum_{c, h,r} a_{s,c}\cdot \delta_{c,d^\prime,h,r} -\beta_{s,d,d^\prime}^\prime \leq 0 + \|\mathbb{C\times R \times H\|}\cdot (1-\sigma_{s,d,d^\prime}) \hspace{1cm} \forall s,d,d^\prime$$

In [392]:
def soft2_part1_rule(model,s,d,d_prim):
  M = len(model.C)*len(model.H)*len(model.R)
  # d_prim += 1 
  if d!=model.lastDay and d_prim == d+1:
    return sum(model.a[s,c]*model.delta[c,d,h,r] for c in model.C for h in model.H for r in model.R) \
        - model.beta[s,d,d_prim] <= M * model.sigma[s,d,d_prim]
  else: 
    return pyo.Constraint.Skip
model.add_component('soft2_part1', pyo.Constraint(model.S,model.D,model.D, rule=soft2_part1_rule))

def soft2_part2_rule(model,s,d,d_prim):
  M = len(model.C)*len(model.H)*len(model.R)
  if d!=model.lastDay and d_prim == d+1:
    return sum(model.a[s,c]*model.delta[c,d_prim,h,r] for c in model.C for h in model.H for r in model.R) \
        - model.beta_prim[s,d,d_prim] <= M * (1 - model.sigma[s,d,d_prim])
  else: 
    return pyo.Constraint.Skip
model.add_component('soft2_part2', pyo.Constraint(model.S,model.D,model.D, rule=soft2_part2_rule))

---
### Soft Constraint 3:


$$\sum_{c} \delta_{c,d,h,r} - \zeta_{d,h,r}\leq 1 \hspace{1cm}\forall d,h,r$$

In [393]:
def soft3_rule(model,d,h,r):
  return sum(model.delta[c,d,h,r] for c in model.C) \
        - model.zeta[d,h,r] <= 1
model.add_component('soft3', pyo.Constraint(model.D,model.H,model.R, rule=soft3_rule))

---

In [394]:
result = pyo.SolverFactory('glpk')
result.options['tmlim'] = 15*60 # LIMIT SOLVER TIME TO 15 MINUTES
result.solve(model,'glpk', tee=True, keepfiles=True)

Solver log file: '/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpixjj9gha.glpk.log'
Solver solution file: '/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmp32xphcxo.glpk.raw'
Solver problem files: ('/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpz569qd2_.pyomo.lp',)
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 900 --write /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmp32xphcxo.glpk.raw
 --wglp /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpwp9pkxdg.glpk.glp
 --cpxlp /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpz569qd2_.pyomo.lp
Reading problem data from '/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpz569qd2_.pyomo.lp'...
/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpz569qd2_.pyomo.lp:4284: warning: lower bound of variable 'x1' redefined
/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpz569qd2_.pyomo.lp:4284: warning: upper bound of variable 'x1' redefined
527 rows, 341 columns, 2241 non-zeros
276 intege

{'Problem': [{'Name': 'unknown', 'Lower bound': 6.6, 'Upper bound': 6.6, 'Number of objectives': 1, 'Number of constraints': 527, 'Number of variables': 341, 'Number of nonzeros': 2241, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '8891', 'Number of created subproblems': '8891'}}, 'Error rc': 0, 'Time': 3.843526840209961}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [395]:
model.display()

Model unknown

  Variables:
    delta : Size=56, Index=delta_index
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0, 0, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 0, 0, 1) :     0 :   0.0 :     1 : False : False : Binary
        (0, 0, 1, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 0, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1, 0, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1, 0, 1) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1, 1, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1, 1, 1) :     0 :   1.0 :     1 : False : False : Binary
        (1, 0, 0, 0) :     0 :   0.0 :     1 : False : False : Binary
        (1, 0, 0, 1) :     0 :   1.0 :     1 : False : False : Binary
        (1, 0, 1, 0) :     0 :   0.0 :     1 : False : False : Binary
        (1, 0, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1, 0, 0) : 

In [396]:
for d in model.D:
    for h in model.H:
        for r in model.R:
            for c in model.C:
                if pyo.value(model.delta[c,d,h,r])==1:
                    sl = [s+1 for s in model.S if pyo.value(model.a[s,c])==1]
                    print(f'({c+1},{d+1},{h+1},{r+1}) : \t 1 \t {sl}')

(4,1,1,1) : 	 1 	 [4]
(2,1,1,2) : 	 1 	 [2, 3]
(3,1,2,1) : 	 1 	 [1, 2, 3]
(3,1,2,2) : 	 1 	 [1, 2, 3]
(7,2,1,1) : 	 1 	 [2, 3, 4]
(7,2,1,2) : 	 1 	 [2, 3, 4]
(5,2,2,1) : 	 1 	 [2]
(1,2,2,2) : 	 1 	 [1]
(6,2,2,2) : 	 1 	 [3]



#### Students
|Course|Student|
|------|-------|
|$c_1$ |$s_1$  |
|$c_2$ |$s_2$, $s_3$|
|$c_3$ |$s_1$, $s_2$, $s_3$|
|$c_4$ |$s_4$  |
|$c_5$ |$s_2$  |
|$c_6$ |$s_3$  |
|$c_7$ |$s_2$, $s_3$, $s_4$ |  

&nbsp;

#### WITHOUT SOFT CONSTRAINT 1 EXTRAS
|             |Room 1 (cap=1)|Room 2(cap=2)|
|-------------|------|------|
|$d_1$ : $h_1$|$c_5$ [$s_2$]|$c_1$ [$s_1$], $c_6$ [$s_3$]|
|$d_1$ : $h_2$|$c_7$ [$s_2$]|$c_7$ [$s_3$, $s_4$]|
|$d_2$ : $h_1$|$c_3$ [$s_1$]|$c_3$ [$s_2$, $s_3$]|
|$d_2$ : $h_2$|$c_4$ [$s_4$]|$c_2$ [$s_2$, $s_3$]|

&nbsp;
&nbsp;

#### WITH SOFT CONSTRAINT 1 EXTRAS
|             |Room 1 (cap=1)|Room 2(cap=2)|
|-------------|------|------|
|$d_1$ : $h_1$|$c_4$ [$s_4$]|$c_2$ [$s_2$, $s_3$]|
|$d_1$ : $h_2$|$c_3$ [$s_1$]|$c_3$ [$s_2$, $s_3$]|
|$d_2$ : $h_1$|$c_7$ [$s_2$]|$c_7$ [$s_3$, $s_4$]|
|$d_2$ : $h_2$|$c_5$ [$s_2$]|$c_1$ [$s_1$],$c_6$ [$s_3$]|
